In [1]:
import os

os.environ["BRASA_DATA_PATH"] = "D:\\brasa"

In [2]:
import sys
sys.path.append('..')

from datetime import datetime

import numpy as np
import pandas as pd
import pyarrow.dataset as ds
import pyarrow.compute as pc
import pyarrow

import brasa

man = brasa.engine.CacheManager()

In [4]:
df = brasa.get_dataset("b3-cotahist")\
    .filter(pc.field("instrument_market") == 10)\
    .filter(pc.field("refdate") >= datetime(2021, 6, 9))\
    .filter(pc.field("refdate") <= datetime(2021, 6, 10))\
    .scanner(["refdate", "symbol", "close", "distribution_id"])\
    .to_table()\
    .to_pandas()
symbols = df.groupby(["symbol"]).apply(lambda x: x.shape[0])
symbols_to_ignore = symbols[symbols == 1].index
df_clean = df[~df["symbol"].isin(symbols_to_ignore)]
symbols = df_clean.groupby(["symbol"]).apply(lambda x: len(x["distribution_id"].unique()))
symbols_to_use = symbols[symbols == 1].index
df_final = df[df["symbol"].isin(symbols_to_use)]
df_final = df_final\
    .groupby(["symbol"])\
    .apply(lambda x: pd.DataFrame([(x.refdate.iloc[1], x.close.iloc[1] / x.close.iloc[0] - 1)], columns=["refdate", "pct_return"]))\
    .reset_index()
df_final = df_final[["refdate", "symbol", "pct_return"]]
df_final["log_return"] = np.log(1 + df_final["pct_return"])
df_final

,refdate,symbol,close,distribution_id
0,2021-06-09,A1AP34,62.51,106
1,2021-06-09,A1BB34,44.00,101
2,2021-06-09,A1CR34,61.02,103
3,2021-06-09,A1EG34,22.92,101
4,2021-06-09,A1EP34,212.94,106
...,...,...,...,...
2144,2021-06-10,GOLL11,7.18,100
2145,2021-06-10,TASA13,18.80,100
2146,2021-06-10,TASA17,17.33,100
2147,2021-06-10,TASA15,17.51,100
